So before we start the actual thing, I will go through a very simple example on how to retrieve an image from OMERO using two different APIs. This is just a teaser for the things we'll be doing, and just to show that it's real easy to do with the scaffolding we've built.

Besides the two APIs we will show, there's a CLI interface that can do a lot of stuff - (almost?) all of it can also be done with the Blitz API, so it would be kinda redundant. But it's useful if you just want to write a bash script to automate something.

So first things first, I'm using a `config.py` file containing all the sensitive info here. In general I have this as a local file added to a .gitignore, make sure I have it accessible only for my system user and don't keep it saved between sessions (or at least keep it saved without the password). In this case I'm using a trainer user on a dev server, with fairly limited permissions, so no reason not to show you.

In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT, OMEROWEB
import warnings
warnings.filterwarnings(action="once") # silence repeated http warnings in JSON API calls

This is just some magic to make matplotlib plot things inline instead of popping up an extra window, and with specific figure format stuff.

In [ ]:
#instantiate qt and wait a bit to make sure

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Ok so first we will do the imports we need here and define which image ID we will show. You have been given some credentials for the dev server, so you should choose one of the images available for you (use OMERO web at ctomeroweb01.jax.org to pick one). Other than that, we import a couple of ezomero helper functions and matplotlib.

In [ ]:
from ezomero import connect
from ezomero import get_image
import matplotlib.pyplot as plt
image_id = 41252

Here we just create a BlitzGateway object with the stuff from our config file and connect it using an ezomero helper function. I'm printing the returned object (anything other than `None` is good). Try running it with fewer arguments to see its behavior in that case!

In [ ]:
conn = connect(OMEROUSER, OMEROPASS, port=OMEROPORT, host=OMEROHOST, group="", secure=True)
print(conn)

Now we use the `conn` object as an argument in our helper function, with image ID and an optional argument specifiying we want XYZCT ordering for dimensions. We always get a 5d numpy array back from this helper function, so we have a look at the shape and then plot the first plane/timepoint/channel (we also need to transpose this to get it seen the same way as in OMERO.web due to how matplotlib interprets coordinate origins and order)

In [ ]:
img, pix = get_image(conn, image_id, xyzct=True)
print(pix.shape)
plt.imshow(pix[:,:,0,0,0].T)

Now we'll do the same thing using the JSONAPI - note that in this case we get a compressed jpeg from the original image, so ideally we should not be using this for quantitative analysis that depends on pixel values. The advantage is that we can use a scale factor to get a downscaled version of the original, which is useful if your original is huge.
We'll just need two helper functions here: create a session and retrieve an image. Creating a session will also use the config file.

In [ ]:
from ezomero.json_api import create_json_session, get_rendered_jpeg

login_rsp, session, base_url = create_json_session(web_host=OMEROWEB, user=OMEROUSER, password=OMEROPASS,
                                                   verify=False) # use verify=False for dev server,
                                                                 # use verify=True for JAX research server

Finally we specify a scale factor for downsampling and then pass it along with image ID and some stuff we got from the session creation to our helper function, and back comes a numpy array again! Try different downsampling factors to see the results.

In [ ]:
scale_factor = 1
img = get_rendered_jpeg(session, base_url, image_id, scale_factor)
plt.imshow(img)